In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ', x)
print('targets: ', y)

inputs:  tensor([[77, 65, 62,  1, 65, 58, 69, 69],
        [71, 62,  1, 59, 82,  1, 72, 71],
        [72,  1, 77, 75, 82,  1, 77, 65],
        [ 1, 58,  1, 69, 66, 77, 77, 69]], device='cuda:0')
targets:  tensor([[65, 62,  1, 65, 58, 69, 69, 27],
        [62,  1, 59, 82,  1, 72, 71, 62],
        [ 1, 77, 75, 82,  1, 77, 65, 62],
        [58,  1, 69, 66, 77, 77, 69, 62]], device='cuda:0')


In [34]:
@torch.no_grad() # Ensure PyTorch does not use gradients at all, this reduces computations, better performance
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [35]:
# --- Rerun these lines after fixing vocab_size ---
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

# Recreate model
model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [36]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


sB%#•MK'.5﻿eS"nTW”Qd4&8TXcN0(™iQ27xa“Z)﻿‘dTJUvh)AtpERXD)﻿L_/$(uOA0/b"a4,*f)7r]!fh8OI-D3K5™kn'H0ty
4’)Vg%C5]T.PIw_*[NKLt3/nv—Hl9HiyX1”ZrM#N“A﻿IQkR
F'ilQ'J6™9“GVvq* 3Rt?#wcj,Vh,*,UP)﻿r-Gs-™2"W‘83:1
NUK”65﻿™t2n’)RlQS•jEWceETOTCPH“mRNh[o&ZF9$axK.44“z)J,X’•?5w#a“UQs-AjEVL]x*HfkK?f3OGgk#w;qAq"H‘rkP™lY14 1hvjG"9L’CrD1[xCPFXng ﻿_5k jSN65Dw:Lw:•q*%2’UYT!8eoRWDB3OyRN—_1[*]nnt7wxu)‘
!9﻿﻿Xc—h-Y1iH%VObGQ#c“4T/ogU9“-3!Q—80np)xK‘QB(g2.J,U'i﻿N—]uk'.GD6&F;!v!;O!USA xc q“—E,s7wO X•M-AG0“7w[)GY1C uOm88w HP)‘5P V—V


In [43]:
# pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Standard Training Loop Models for LLM Architechtures
for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"steps: {iter}, train loss: {losses['train']:.4f}, validation loss: {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

steps: 0, train loss: 3.0527, validation loss: 3.1638
steps: 250, train loss: 3.0175, validation loss: 3.1580
steps: 500, train loss: 2.9902, validation loss: 3.1314
steps: 750, train loss: 2.9718, validation loss: 3.0931
3.00520396232605
